# Build and Push Docker Image

This tutorial will guide you through the process of building and pushing a Docker image to the MAIA Docker registry.
The Docker image will be remotely built on the cloud and pushed to the MAIA Docker registry, which is hosted at `registry.cloud.cbh.kth.se`.
The Docker Context (Dockerfile and other necessary files) need to be located in a Git repository, which will be accessed by [KANIKO](https://github.com/GoogleContainerTools/kaniko) to build the Docker image.

The configuration parameters for building and pushing the Docker image are set in a JSON configuration file, following the conventions described below.
```yaml
git_path: "<GIT_PATH>"              # The path to the Git repository where the Docker Context is located
git_subpath: "<GIT_PATH>"          # The subpath to the Docker Context in the Git repository
git_username: "<GIT_USERNAME>"    # The username to access the Git repository
git_token: "<GIT_TOKEN>"           # The token to access the Git repository
docker_registry_secret: "<SECRET>" # Kubernetes Secret to Authenticate in the private Registry
docker_image: "<IMAGE_NAME>"       # The name of the Docker image to build
buildArgs:                         # Optional build arguments
    OPTIONAL_BUILD_ARG: "<VALUE>"
    OPTIONAL_BUILD_ARG: "<VALUE>"
```

## Set Git Specifications

In this example, we will build our custom Jupyter Scipy Image, where it will be possible to set the Jupyter Token as an env variable `PW`.
For more details, check the [Dockerfile](https://github.com/kthcloud/MAIA/blob/master/docker/demo/Dockerfile)

In [ ]:
%%writefile kaniko_values.yaml

docker_registry_secret: <SECRET>
pvc:
  pvc_type: <STORAGE_CLASS>
  size: "10Gi"

args:
  - "--dockerfile=Dockerfile"
  - "--context=git://github.com/kthcloud/MAIA.git"
  - "--context-sub-path=docker/demo"
  - "--destination=registry.cloud.cbh.kth.se/maia/jupyter-demo:1.2"
  - "--cache=true"
  - "--cache-dir=/cache"
  #- "--context=s3://docker-contexts/{{.Values.context_file}}"


## Install Helm Chart

In [ ]:
%%bash

helm repo add maia https://kthcloud.github.io/MAIA/
helm repo update
export KUBECONFIG=<KUBECONFIG>

#Optional: delete the previous deployment
# helm delete -n maia-admin demo-docker

helm install -n maia-admin demo-docker maia/mkg-kaniko --values kaniko_values.yaml --version 1.0.3

## Log the Docker Build Job

In [ ]:
%%bash

export KUBECONFIG=<KUBECONFIG>

kubectl logs -f -n maia-admin job/demo-docker-mkg-kaniko